### Physical Damage vs. Magic Damage: How much do you want in your team?

In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append('../src/')
import get_modeling_data
import data_constants as dc

In [2]:
# Load match data
data = get_modeling_data.get_all_data()

In [3]:
print(data.shape)
data.head()

(285536, 15)


,match_id,game_version,queue_id,game_duration,team_100_win,100_TOP_SOLO,100_JUNGLE_NONE,100_MIDDLE_SOLO,100_BOTTOM_DUO_CARRY,100_BOTTOM_DUO_SUPPORT,200_TOP_SOLO,200_JUNGLE_NONE,200_MIDDLE_SOLO,200_BOTTOM_DUO_CARRY,200_BOTTOM_DUO_SUPPORT
0,2726806213,8.4.218.8787,420,1452,0,Kled,Reng,Kata,Varu,Tari,Akal,Vi,Dr.,Vayn,Lulu
1,2726789592,8.4.218.8787,420,1539,0,Rene,Mast,Fizz,Cait,Lux,Kled,Skar,Kass,Ezre,Sona
2,2726784050,8.4.218.8787,420,1380,0,Kled,Reng,Ryze,Varu,Sora,Gang,Seju,Ahri,Jinx,Blit
3,2726032902,8.4.218.8787,420,1360,1,Kled,Kayn,Fizz,Varu,Tari,Blit,Shac,Kayl,Tris,Lux
4,2726003824,8.4.218.8787,420,1773,1,Kled,Lee,Aniv,Drav,Bard,Yasu,Reng,Ekko,Tris,Tahm


In [4]:
# Load lookup table
phys_magic_lut = pd.read_csv('../data/processed/champs_phys_magic_ind.csv')
phys_magic_lut.head()

,champ,Phys,Magic,Neither
0,Aatr,1.0,NaN,NaN
1,Ahri,NaN,1.0,NaN
2,Akal,NaN,1.0,NaN
3,Alis,NaN,NaN,1.0
4,Amum,NaN,1.0,NaN


In [5]:
phys_magic_lut = phys_magic_lut.fillna(0)
print("Total Physical Dmg Dealers: " + str(np.sum(phys_magic_lut.Phys)))
print("Total Magic Dmg Dealers: " + str(np.sum(phys_magic_lut.Magic)))
print("Total Neither Dmg Dealers: " + str(np.sum(phys_magic_lut.Neither)))

Total Physical Dmg Dealers: 63.0
Total Magic Dmg Dealers: 59.0
Total Neither Dmg Dealers: 18.0


In [6]:
# Calculate number of phys and number of magical on each team
teams_lanes_roles = dc.get_teams_lanes_roles()
teams = dc.get_teams()
lanes_roles = dc.get_lanes_roles()

    # Join individ win rates
for tlr in teams_lanes_roles:
    print(tlr)
    # want to take index of indiv win rates and a single column of it
    data = pd.merge(data, phys_magic_lut, how='left', left_on=tlr, right_on='champ',
                   suffixes=('','_'+tlr))
    #data = data.rename({tlr[4:] + '_win_rate': tlr + '_wr'}, axis=1)
    data = data.drop(['champ'], axis=1)
data = data.rename({'Phys':'Phys_100_TOP_SOLO',
                    'Magic':'Magic_100_TOP_SOLO',
                   'Neither':'Neither_100_TOP_SOLO'}, axis=1)

100_TOP_SOLO
100_MIDDLE_SOLO
100_BOTTOM_DUO_CARRY
100_BOTTOM_DUO_SUPPORT
100_JUNGLE_NONE
200_TOP_SOLO
200_MIDDLE_SOLO
200_BOTTOM_DUO_CARRY
200_BOTTOM_DUO_SUPPORT
200_JUNGLE_NONE


In [7]:
data.columns

Index(['match_id', 'game_version', 'queue_id', 'game_duration', 'team_100_win',
       '100_TOP_SOLO', '100_JUNGLE_NONE', '100_MIDDLE_SOLO',
       '100_BOTTOM_DUO_CARRY', '100_BOTTOM_DUO_SUPPORT', '200_TOP_SOLO',
       '200_JUNGLE_NONE', '200_MIDDLE_SOLO', '200_BOTTOM_DUO_CARRY',
       '200_BOTTOM_DUO_SUPPORT', 'Phys_100_TOP_SOLO', 'Magic_100_TOP_SOLO',
       'Neither_100_TOP_SOLO', 'Phys_100_MIDDLE_SOLO', 'Magic_100_MIDDLE_SOLO',
       'Neither_100_MIDDLE_SOLO', 'Phys_100_BOTTOM_DUO_CARRY',
       'Magic_100_BOTTOM_DUO_CARRY', 'Neither_100_BOTTOM_DUO_CARRY',
       'Phys_100_BOTTOM_DUO_SUPPORT', 'Magic_100_BOTTOM_DUO_SUPPORT',
       'Neither_100_BOTTOM_DUO_SUPPORT', 'Phys_100_JUNGLE_NONE',
       'Magic_100_JUNGLE_NONE', 'Neither_100_JUNGLE_NONE', 'Phys_200_TOP_SOLO',
       'Magic_200_TOP_SOLO', 'Neither_200_TOP_SOLO', 'Phys_200_MIDDLE_SOLO',
       'Magic_200_MIDDLE_SOLO', 'Neither_200_MIDDLE_SOLO',
       'Phys_200_BOTTOM_DUO_CARRY', 'Magic_200_BOTTOM_DUO_CARRY',
       'Neit

In [8]:
data['100_Phys_Total'] = data['Phys_100_TOP_SOLO'] + data['Phys_100_JUNGLE_NONE'] + data['Phys_100_MIDDLE_SOLO'] + data['Phys_100_BOTTOM_DUO_CARRY'] + data['Phys_100_BOTTOM_DUO_SUPPORT']
data['100_Magic_Total'] = data['Magic_100_TOP_SOLO'] + data['Magic_100_JUNGLE_NONE'] + data['Magic_100_MIDDLE_SOLO'] + data['Magic_100_BOTTOM_DUO_CARRY'] + data['Magic_100_BOTTOM_DUO_SUPPORT']
data['100_Neither_Total'] = data['Neither_100_TOP_SOLO'] + data['Neither_100_JUNGLE_NONE'] + data['Neither_100_MIDDLE_SOLO'] + data['Neither_100_BOTTOM_DUO_CARRY'] + data['Neither_100_BOTTOM_DUO_SUPPORT']
data['200_Phys_Total'] = data['Phys_200_TOP_SOLO'] + data['Phys_200_JUNGLE_NONE'] + data['Phys_200_MIDDLE_SOLO'] + data['Phys_200_BOTTOM_DUO_CARRY'] + data['Phys_200_BOTTOM_DUO_SUPPORT']
data['200_Magic_Total'] = data['Magic_200_TOP_SOLO'] + data['Magic_200_JUNGLE_NONE'] + data['Magic_200_MIDDLE_SOLO'] + data['Magic_200_BOTTOM_DUO_CARRY'] + data['Magic_200_BOTTOM_DUO_SUPPORT']
data['200_Neither_Total'] = data['Neither_200_TOP_SOLO'] + data['Neither_200_JUNGLE_NONE'] + data['Neither_200_MIDDLE_SOLO'] + data['Neither_200_BOTTOM_DUO_CARRY'] + data['Neither_200_BOTTOM_DUO_SUPPORT']

In [9]:
data['100_Phys_Total'].describe()

count    285476.000000
mean          2.602324
std           0.720223
min           0.000000
25%           2.000000
50%           3.000000
75%           3.000000
max           5.000000
Name: 100_Phys_Total, dtype: float64

In [10]:
data['100_Magic_Total'].describe()

count    285476.000000
mean          1.663313
std           0.810927
min           0.000000
25%           1.000000
50%           2.000000
75%           2.000000
max           5.000000
Name: 100_Magic_Total, dtype: float64

In [11]:
# Calculate phys win rates
phys_win_rates = {}
for i in range(1,6):
    games_won = np.sum(data['team_100_win'][data['100_Phys_Total'] == i]) + np.sum(1 - data['team_100_win'][data['200_Phys_Total'] == i])
    games_played = np.sum(data['100_Phys_Total'] == i) + np.sum(data['200_Phys_Total'] == i)
    phys_win_rates[str(i)] = games_won / games_played
print(phys_win_rates)

{'1': 0.46998662107923295, '2': 0.4998489985192113, '3': 0.5074147673273315, '4': 0.4774432450747858, '5': 0.3872502378686965}


In [12]:
# Calculate magic win rates
magic_win_rates = {}
for i in range(1,6):
    games_won = np.sum(data['team_100_win'][data['100_Magic_Total'] == i]) + np.sum(1 - data['team_100_win'][data['200_Magic_Total'] == i])
    games_played = np.sum(data['100_Magic_Total'] == i) + np.sum(data['200_Magic_Total'] == i)
    magic_win_rates[str(i)] = games_won / games_played
print(magic_win_rates)

{'1': 0.508253582238396, '2': 0.5011994105753744, '3': 0.48581786311847663, '4': 0.44284341978866476, '5': 0.38571428571428573}


In [13]:
# Calculate blue phys win rates
phys_win_rates = {}
for i in range(1,6):
    games_won = np.sum(data['team_100_win'][data['100_Phys_Total'] == i])
    games_played = np.sum(data['100_Phys_Total'] == i)
    phys_win_rates[str(i)] = games_won / games_played
print(phys_win_rates)

{'1': 0.4627044515008088, '2': 0.49458880835935815, '3': 0.5015258288729533, '4': 0.47062725033162783, '5': 0.3980952380952381}


In [14]:
# Calculate blue magic win rates
magic_win_rates = {}
for i in range(1,6):
    games_won = np.sum(data['team_100_win'][data['100_Magic_Total'] == i])
    games_played = np.sum(data['100_Magic_Total'] == i)
    magic_win_rates[str(i)] = games_won / games_played
print(magic_win_rates)

{'1': 0.5023058008173124, '2': 0.49561065102961094, '3': 0.4801321855928364, '4': 0.4424559471365639, '5': 0.2647058823529412}


In [20]:
# Number of magic i games
for i in range(1,6):
    print(np.sum(data['100_Magic_Total'] == i)/data.shape[0])

0.40021923680376553
0.4085159139302925
0.13141250140087415
0.012719937240838284
0.00011907430236467555


In [21]:
# Number of phys i games
for i in range(1,6):
    print(np.sum(data['100_Phys_Total'] == i)/data.shape[0])

0.0584549759049647
0.35661352684074865
0.508401742687437
0.07392412865628152
0.0018386473159251373


In [25]:
# Number of magic i games
for i in range(1,6):
    print(np.sum(data['team_100_win'][np.logical_and(data['100_Magic_Total'] == i, data['200_Phys_Total'] == i)]))

3622
21073
8982
132
0


In [26]:
# Number of magic i games
for i in range(1,6):
    print(np.sum(np.logical_and(data['100_Magic_Total'] == i, data['200_Phys_Total'] == i)))

6865
42214
19150
286
0


In [30]:
132/286

0.46153846153846156